In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
test_df = pd.read_csv("/content/drive/MyDrive/dataset/test.csv")

In [4]:
test_df

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318
...,...,...,...,...,...
734731,921419,"Casual Canine Basic Hoodie for Dogs, 16"" Mediu...",[Brightly colored pet sweatshirts with authent...,"ZA6015 16 43 Size-See Chart Below: Medium: 16""...",7073
734732,2456362,Dive Log Book: Scuba Diving Logbook for Beginn...,NaN,NaN,1
734733,841529,Axor 39135001 Citterio Widespread Faucet with ...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo...",39135001 Features: -ADA compliant. -Includes p...,10645
734734,1190194,Caroline's Treasures BB1801DS812 Halloween Bas...,"[Indoor or outdoor aluminum artwork prints,8 i...",Features. Great for inside or outside these Al...,12680


In [5]:
train_df = pd.read_csv("/content/drive/MyDrive/dataset/train.csv")

In [6]:
train_df

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000
...,...,...,...,...,...,...
2249693,2422167,Nike Women's As W Ny Df Swsh Hn Kh Bra (CZ7610...,Material : Polyester,NaN,3009,1181.100000
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[❤ [Inspiration] Inspired by the Untitled Goos...,<p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...,3413,125.984252
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,"[Dial size: 12 inches in diameter,Big, clear r...",Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000
2249696,1165754,Electro Voice EKX-BRKT15 | Wall Mount Bracket ...,NaN,NaN,592,2900.000000


In [7]:
train_df = train_df.sample(n=3000, random_state=42)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Input,
    Embedding,
    Dense,
    Conv1D,
    MaxPooling1D,
    LSTM,
    Bidirectional,
    concatenate,
    Dropout,
)
from tensorflow.keras.models import Model

In [9]:
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
train_df['TITLE'] = train_df['TITLE'].astype(str)
train_df['DESCRIPTION'] = train_df['DESCRIPTION'].astype(str)
train_df['BULLET_POINTS'] = train_df['BULLET_POINTS'].astype(str)
tokenizer.fit_on_texts(list(train_df['TITLE'].values) + list(train_df['DESCRIPTION'].values) + list(train_df['BULLET_POINTS'].values))
maxlen = 300

In [10]:
X_train_title = tokenizer.texts_to_sequences(train_df['TITLE'].values)
X_train_desc = tokenizer.texts_to_sequences(train_df['DESCRIPTION'].values)
X_train_bull = tokenizer.texts_to_sequences(train_df['BULLET_POINTS'].values)
X_train_title = pad_sequences(X_train_title, maxlen=maxlen)
X_train_desc = pad_sequences(X_train_desc, maxlen=maxlen)
X_train_bull = pad_sequences(X_train_bull, maxlen=maxlen)

In [11]:
type_train = train_df['PRODUCT_TYPE_ID'].values.reshape(-1, 1)
type_test = test_df['PRODUCT_TYPE_ID'].values.reshape(-1, 1)

In [12]:
input1 = Input(shape=(maxlen,))
x1 = Embedding(max_features, 64)(input1)
x1 = Conv1D(filters=16, kernel_size=3, padding='same', activation='relu')(x1)
x1 = MaxPooling1D(pool_size=2)(x1)
x1 = Bidirectional(LSTM(32, return_sequences=True))(x1)
x1 = Bidirectional(LSTM(16))(x1)
input2 = Input(shape=(1,))
x2 = Dense(8, activation='relu')(input2)

In [13]:
concat = concatenate([x1, x2])
output = Dense(1)(concat)
model = Model(inputs=[input1, input2], outputs=output)
model.compile(loss='mse', optimizer='adam')

In [14]:
batch_size = 16
epochs = 3
num_train_samples = X_train_title.shape[0]
num_train_steps = num_train_samples // batch_size

In [15]:
history = model.fit(
    [X_train_title, type_train],
    train_df['PRODUCT_LENGTH'].values,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    steps_per_epoch=num_train_steps,
    verbose=1
)

Epoch 1/3
187/187 [==============================] - 49s 208ms/step - loss: 23063019520.0000 - val_loss: 3571136.5000
Epoch 2/3
187/187 [==============================] - 35s 188ms/step - loss: 22441551872.0000 - val_loss: 3685482.2500
Epoch 3/3
 76/187 [===========>..................] - ETA: 20s - loss: 53586460672.0000

187/187 [==============================] - 16s 85ms/step - loss: 53586460672.0000 - val_loss: 3826808.2500


In [16]:
train_df = train_df.fillna('')
test_df= test_df.fillna('')
train_title_seq = tokenizer.texts_to_sequences(train_df['TITLE'].values)
train_desc_seq = tokenizer.texts_to_sequences(train_df['DESCRIPTION'].values)
train_bullet_seq = tokenizer.texts_to_sequences(train_df['BULLET_POINTS'].values)

In [17]:
test_title_seq = tokenizer.texts_to_sequences(test_df['TITLE'].values)
test_desc_seq = tokenizer.texts_to_sequences(test_df['DESCRIPTION'].values)
test_bullet_seq = tokenizer.texts_to_sequences(test_df['BULLET_POINTS'].values)
X_train_title = pad_sequences(train_title_seq, maxlen=maxlen)
X_train_desc = pad_sequences(train_desc_seq, maxlen=maxlen)
X_train_bullet = pad_sequences(train_bullet_seq, maxlen=maxlen)

In [18]:
X_test_title = pad_sequences(test_title_seq, maxlen=maxlen)
X_test_desc = pad_sequences(test_desc_seq, maxlen=maxlen)
X_test_bullet = pad_sequences(test_bullet_seq, maxlen=maxlen)
X_train = np.concatenate((X_train_title, X_train_desc, X_train_bullet), axis=1)
X_test = np.concatenate((X_test_title, X_test_desc, X_test_bullet), axis=1)

In [ ]:
y_pred = model.predict([X_test, type_test])

In [ ]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df['PRODUCT_LENGTH'] = y_pred

In [ ]:
submission_df